In [ ]:
!ls
%cd ../..
!ls
# Now you should see the bubble-segmentation-final-deep-learning directory

In [ ]:
import os
import subprocess
from pathlib import Path

END_WITH_LOCAL = 'bubble-segmentation-final-deep-learning'

os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"

BASE_DIR = os.getcwd()
print(f"BASE_DIR: {BASE_DIR}")

# Simple validation
if not (BASE_DIR.endswith('/content') or BASE_DIR.endswith(END_WITH_LOCAL)):
    raise ValueError(f"Expected to be in .../{END_WITH_LOCAL} or .../content directory, but got: {BASE_DIR}")

In [ ]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")


In [ ]:
import os
import json
import xml.etree.ElementTree as ET
from tqdm import tqdm
import random

# --- Configuration ---
base_path = os.path.join(BASE_DIR, "data", "Manga109_released_2023_12_07")  # Adjusted to the new dataset directory
annotations_dir = os.path.join(base_path, "annotations")
CATEGORIES_TO_DETECT = ["body", "frame"]
VAL_SPLIT_RATIO = 0.2  # Use 20% of the data for validation

def process_books(book_list, output_filename):
    """Processes a list of books and saves them to a COCO JSON file."""
    
    # Category mapping (IDs should start from 1)
    categories_coco = [{"id": i, "name": name} for i, name in enumerate(CATEGORIES_TO_DETECT, start=1)]
    category_map = {name: i for i, name in enumerate(CATEGORIES_TO_DETECT, start=1)}
    
    coco_output = {
        "info": {
            "description": "Manga109 Dataset for Detectron2",
            "version": "1.0",
        },
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": categories_coco
    }
    
    annotation_id_counter = 1
    image_id_counter = 1
    
    print(f"Processing {len(book_list)} books for '{output_filename}'...")
    for book in tqdm(book_list, desc=f"Processing for {output_filename}"):
        xml_file_path = os.path.join(annotations_dir, f"{book}.xml")
        
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        
        for page in root.findall(".//page"):
            page_index = page.attrib['index']
            image_width = int(page.attrib['width'])
            image_height = int(page.attrib['height'])
            
            image_rel_path = os.path.join(book, f"{page_index.zfill(3)}.jpg")
            
            image_info = {
                "id": image_id_counter,
                "file_name": image_rel_path,
                "width": image_width,
                "height": image_height,
            }
            coco_output["images"].append(image_info)

            for category_name in CATEGORIES_TO_DETECT:
                for obj in page.findall(category_name):
                    bbox = obj.attrib
                    xmin = int(bbox['xmin'])
                    ymin = int(bbox['ymin'])
                    width = int(bbox['xmax']) - xmin
                    height = int(bbox['ymax']) - ymin
                    
                    annotation_info = {
                        "id": annotation_id_counter,
                        "image_id": image_id_counter,
                        "category_id": category_map[category_name],
                        "bbox": [xmin, ymin, width, height],
                        "area": width * height,
                        "iscrowd": 0,
                    }
                    coco_output["annotations"].append(annotation_info)
                    annotation_id_counter += 1
            
            image_id_counter += 1

    output_json_path = os.path.join(base_path, output_filename)
    with open(output_json_path, 'w') as f:
        json.dump(coco_output, f)
        
    print(f"\nConversion complete! File saved to: {output_json_path}")
    return output_json_path

SEED = 42
random.seed(SEED)

if __name__ == '__main__':
    all_books = [f.replace('.xml', '') for f in os.listdir(annotations_dir) if f.endswith('.xml')]
    random.shuffle(all_books) # Shuffle for a random split

    split_index = int(len(all_books) * VAL_SPLIT_RATIO)
    val_books = all_books[:split_index]
    train_books = all_books[split_index:]
    
    print(f"Total books: {len(all_books)}. Training books: {len(train_books)}. Validation books: {len(val_books)}.")

    # Process and save the training and validation sets
    process_books(train_books, "train.json")
    process_books(val_books, "val.json")

In [ ]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
import random
import cv2
from detectron2.utils.visualizer import Visualizer
import matplotlib.pyplot as plt

# --- Paths ---
base_path = os.path.join(BASE_DIR, "data", "Manga109_released_2023_12_07")  # Adjusted to the new dataset directory
images_dir = os.path.join(base_path, "images")
train_json_path = os.path.join(base_path, "train.json")
val_json_path = os.path.join(base_path, "val.json")
CATEGORIES_TO_DETECT = ["body", "frame"]

# --- Register the dataset ---
# The first argument is the name you will use to refer to this dataset.
# Register the training dataset
register_coco_instances("manga109_train", {}, train_json_path, images_dir)

# Register the validation dataset
register_coco_instances("manga109_val", {}, val_json_path, images_dir)

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2 import model_zoo
from detectron2.config import get_cfg

# --- Configuration Setup ---
cfg = get_cfg()

# Start with a pre-trained model. Faster R-CNN is a good choice.
# Find more models in the Detectron2 Model Zoo: https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# cfg.MODEL.WEIGHTS = os.path.join(BASE_DIR, "models/bubble-detection/detectron2/model_final_280758.pkl")
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

cfg.MODEL.DEVICE = str(device)

# --- Dataset Configuration ---
cfg.DATASETS.TRAIN = ("manga109_train",)
# --- ADD THE TEST DATASET ---
cfg.DATASETS.TEST = ("manga109_val",) 
# ----------------------------

# --- Evaluation Configuration ---
# How often to run evaluation (in number of iterations).
cfg.TEST.EVAL_PERIOD = 500 
# --------------------------------

cfg.DATALOADER.NUM_WORKERS = 2 # 12
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # body, frame

cfg.SOLVER.IMS_PER_BATCH = 2 # 8
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 3000 
cfg.SOLVER.STEPS = []

cfg.OUTPUT_DIR = os.path.join(BASE_DIR, "models", "bubble-detection","detectron2")
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# --- Start Training ---
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer, ColorMode

# --- Setup the predictor ---
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # Path to your trained model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # Set a threshold for predictions
predictor = DefaultPredictor(cfg)

# Get the metadata and the list of data dictionaries for the validation set
val_metadata = MetadataCatalog.get("manga109_val")
dataset_dicts = DatasetCatalog.get("manga109_val")

# Randomly select a few images from the validation set to visualize
num_images_to_show = 3
for d in random.sample(dataset_dicts, num_images_to_show):    
    im = cv2.imread(d["file_name"])
    if im is None:
        print(f"Warning: Could not read image {d['file_name']}. Skipping.")
        continue

    # Make predictions
    outputs = predictor(im)

    # --- Visualize Predictions ---
    v_pred = Visualizer(im[:, :, ::-1],
                   metadata=val_metadata, 
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE_BW # or ColorMode.IMAGE
    )
    predictions_vis = v_pred.draw_instance_predictions(outputs["instances"].to("cpu"))

    # --- Visualize Ground Truth ---
    v_gt = Visualizer(im[:, :, ::-1],
                   metadata=val_metadata, 
                   scale=0.8
    )
    ground_truth_vis = v_gt.draw_dataset_dict(d)

    # --- Display side-by-side ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    
    ax1.imshow(predictions_vis.get_image()[:, :, ::-1])
    ax1.set_title("Model Predictions")
    ax1.axis("off")

    ax2.imshow(ground_truth_vis.get_image()[:, :, ::-1])
    ax2.set_title("Ground Truth")
    ax2.axis("off")
    
    plt.tight_layout()
    plt.show()